In [45]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [46]:
tf.reset_default_graph()

In [47]:
z_dim = 10
h_dim=128
mb_size = 32

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

def sample_z(n,m):
    return np.random.uniform(-1.,1., size=[n,m])

In [48]:
# Defining input placeholder and parameters for the generator

Z = tf.placeholder(tf.float32, shape=[None, z_dim])
with tf.name_scope("generator"):
    G_W1 = tf.Variable(xavier_init([z_dim, h_dim]),name="G_W1")
    G_b1 = tf.Variable(tf.zeros(shape=[h_dim]),name="G_B1")

    G_W2 = tf.Variable(xavier_init([h_dim, 784]),name="G_W2")
    G_b2 = tf.Variable(tf.zeros(shape=[784]),name="G_B2")

theta_G = [G_W1, G_W2, G_b1, G_b2]

In [49]:
# Defining input placeholder and parameters for the discriminator

X = tf.placeholder(tf.float32, shape=[None, 784])

with tf.name_scope("discriminator"):

    D_W1 = tf.Variable(xavier_init([784, h_dim]), name="D_W1")
    D_b1 = tf.Variable(tf.zeros(shape=[h_dim]),name="D_B1")

    D_W2 = tf.Variable(xavier_init([128, 1]),name="D_W2")
    D_b2 = tf.Variable(tf.zeros(shape=[1]),name="D_B2")

theta_D = [D_W1, D_W2, D_b1, D_b2]

In [50]:
def generator(z):
    with tf.name_scope("generator"):
        G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
        G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
        G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob

In [51]:
def discriminator(x):
    with tf.name_scope("discriminator"):
        D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
        D_out = tf.matmul(D_h1, D_W2) + D_b2
    return D_out

In [52]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [53]:
G_sample = generator(Z)
D_real = discriminator(X)
D_fake = discriminator(G_sample)

In [54]:
D_loss = tf.reduce_mean(D_real) - tf.reduce_mean(D_fake)
G_loss = -tf.reduce_mean(D_fake)

In [55]:
D_solver = (tf.train.RMSPropOptimizer(learning_rate=5e-5)
            .minimize(-D_loss, var_list=theta_D))
G_solver = (tf.train.RMSPropOptimizer(learning_rate=5e-5)
            .minimize(G_loss, var_list=theta_G))

clip_D = [p.assign(tf.clip_by_value(p, -0.01, 0.01)) for p in theta_D]

In [56]:
mnist = input_data.read_data_sets('../data/MNIST_data', one_hot=True)

Extracting ../data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [57]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())

In [44]:
if not os.path.exists('wgan_out/'):
    os.makedirs('wgan_out/')

i = 0

for it in range(1000000):
    if it % 100 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_z(16, z_size)})

        fig = plot(samples)
        plt.savefig('wgan_out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)
    
    for _ in range(5):

        X_mb, _ = mnist.train.next_batch(mb_size)

        _, D_loss_curr, _ = sess.run([D_solver, D_loss, clip_D], feed_dict={X: X_mb, Z: sample_z(mb_size, z_size)})
    
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_z(mb_size, z_size)})

    if it % 100 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

Iter: 0
D loss: -0.0005289
G_loss: -0.001736

Iter: 100
D loss: 1.576
G_loss: 1.283

Iter: 200
D loss: 1.626
G_loss: 1.242

Iter: 300
D loss: 1.497
G_loss: 1.12

Iter: 400
D loss: 1.375
G_loss: 1.004

Iter: 500
D loss: 1.261
G_loss: 0.8513

Iter: 600
D loss: 1.135
G_loss: 0.7208

Iter: 700
D loss: 1.076
G_loss: 0.6877

Iter: 800
D loss: 0.9044
G_loss: 0.7019

Iter: 900
D loss: 0.808
G_loss: 0.6899

Iter: 1000
D loss: 0.6514
G_loss: 0.6608

Iter: 1100
D loss: 0.5789
G_loss: 0.5616

Iter: 1200
D loss: 0.4553
G_loss: 0.5413

Iter: 1300
D loss: 0.3537
G_loss: 0.4698

Iter: 1400
D loss: 0.3292
G_loss: 0.4525

Iter: 1500
D loss: 0.1889
G_loss: 0.3163

Iter: 1600
D loss: 0.2064
G_loss: 0.229

Iter: 1700
D loss: 0.1307
G_loss: 0.1591

Iter: 1800
D loss: 0.09894
G_loss: 0.138

Iter: 1900
D loss: 0.07232
G_loss: 0.06715

Iter: 2000
D loss: 0.05673
G_loss: 0.06154

Iter: 2100
D loss: 0.05363
G_loss: 0.04845



KeyboardInterrupt: 